In [1]:
import sys
import os
import json

import numpy as np
import pandas as pd
from tqdm import tqdm

# Инструменты Detectron2
import detectron2
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultTrainer, default_setup, launch
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger

/home/dtsarev/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from detectron2.projects.deeplab import add_deeplab_config

In [3]:
DATASET_PATH = '/home/dtsarev/master_of_cv/sem3/DL_project/data/splits_final_deblurred/'
WEIGHTS = '/home/dtsarev/master_of_cv/sem3/DL_project/weights/deeplab_model_final.pth'
RESULTS = '/home/dtsarev/master_of_cv/sem3/DL_project/data/results/'

In [4]:
sys.path.insert(0, os.path.abspath('..'))
from deeplab.train_net import (
    setup as deeplab_setup,  # ваша функция setup(cfg)
    Trainer as DeeplabTrainer, 
    register_zero_waste_semseg,
)

logger = setup_logger()

In [5]:
labels_json = "/home/dtsarev/master_of_cv/sem3/DL_project/data/splits_final_deblurred/train/labels.json"
with open(labels_json, 'r') as f:
    data = json.load(f)

In [6]:
import glob
from skimage import io

sem_seg_dir = "/home/dtsarev/master_of_cv/sem3/DL_project/data/splits_final_deblurred/train/sem_seg/*.PNG"
all_masks = glob.glob(sem_seg_dir)

# Инициализируем счётчик на 5 классов
NUM_CLASSES = 5
pixel_count = np.zeros(NUM_CLASSES, dtype=np.uint64)

for mask_path in tqdm(all_masks, desc="Counting pixels"):
    mask = io.imread(mask_path)
    # mask.shape = (H, W), пиксели 0..4 или 255(игнор)
    # Подсчитаем встречаемость каждого значения
    vals, counts = np.unique(mask, return_counts=True)
    for v, c in zip(vals, counts):
        if v < NUM_CLASSES:
            pixel_count[v] += c
        # если v==255 => ignore

print("Pixel distribution:", pixel_count)

Counting pixels: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3002/3002 [00:39<00:00, 75.90it/s]

Pixel distribution: [5213096699   39188803  645025935    5711901  321923862]


In [7]:
total_pixels = pixel_count.sum()
class_ratio = pixel_count / total_pixels  # доля пикселей каждого класса
epsilon = 1e-6
class_weight = 1.0 / (class_ratio + epsilon)

# Опционально нормализуем, чтобы средний вес был 1
class_weight = class_weight / class_weight.mean()

print("Class weights:", class_weight)

Class weights: [0.00467307 0.6215383  0.03776744 4.26034882 0.07567237]


In [8]:
data_root = "/home/dtsarev/master_of_cv/sem3/DL_project/data/splits_final_deblurred/"
register_zero_waste_semseg(data_root)

# Загружаем базовую конфигурацию из zerowaste_config.yaml
cfg = get_cfg()
add_deeplab_config(cfg)
cfg.merge_from_file("../deeplab/configs/zerowaste_config.yaml")

cfg.MODEL.WEIGHTS = WEIGHTS

# Включаем auxiliary loss (если не включено)
#cfg.MODEL.SEM_SEG_HEAD.AUX_LOSS = True

# Прописываем веса для loss
cfg.MODEL.SEM_SEG_HEAD.CLASS_WEIGHT = class_weight.tolist()

# При желании изменить некоторые hyperparams (например, снизить LR):
cfg.SOLVER.BASE_LR = 0.001

Registering the zero-waste dataset splits


In [9]:
cfg.freeze()
default_setup(cfg, None)  # инициализирует логгер и пр.

[12/21 19:39:39 detectron2]: Rank of current process: 0. World size: 1
[12/21 19:39:39 detectron2]: Rank of current process: 0. World size: 1
[12/21 19:39:39 detectron2]: Environment info:
-------------------------------  ------------------------------------------------------------------------------
sys.platform                     linux
Python                           3.11.4 (main, Jul  5 2023, 14:15:25) [GCC 11.2.0]
numpy                            1.24.2
detectron2                       0.6 @/home/dtsarev/anaconda3/lib/python3.11/site-packages/detectron2
Compiler                         GCC 11.4
CUDA compiler                    not available
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.3.1+cu121 @/home/dtsarev/anaconda3/lib/python3.11/site-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_ABI  False
GPU available                    Yes
GPU 0                            NVIDIA GeForce RTX 4070 (arch=8.9)
Driver version 

In [10]:
trainer = DeeplabTrainer(cfg)
trainer.resume_or_load(resume=False)  # False => начать с нуля (но с предтренированными весами)

[12/21 19:39:47 d2.engine.defaults]: Model:
SemanticSegmentor(
  (backbone): ResNet(
    (stem): DeepLabStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv3): Conv2d(
        64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=128, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e

In [11]:
trainer.train()

[12/21 19:39:56 d2.engine.train_loop]: Starting training from iteration 0
[12/21 19:39:56 d2.engine.train_loop]: Starting training from iteration 0
ERROR [12/21 19:39:59 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/home/dtsarev/anaconda3/lib/python3.11/site-packages/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/home/dtsarev/anaconda3/lib/python3.11/site-packages/detectron2/engine/defaults.py", line 498, in run_step
    self._trainer.run_step()
  File "/home/dtsarev/anaconda3/lib/python3.11/site-packages/detectron2/engine/train_loop.py", line 310, in run_step
    loss_dict = self.model(data)
                ^^^^^^^^^^^^^^^^
  File "/home/dtsarev/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dtsarev/anaconda3/lib/python3.11/site-packages/torch/nn/mod

OutOfMemoryError: CUDA out of memory. Tried to allocate 456.00 MiB. GPU 